# Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import NoteComposeNet
from dataset import MidiDataset
from torch.utils.data import DataLoader
from train import TrainPipeline

import torch
import pandas as pd

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [3]:
BATCH_SIZE = 32
TRAIN_SAMPLES_PER_TRACK = 1
VALIDATE_SAMPLES_PER_TRACK = 1
EPOCHS = 8

In [4]:
CHECKPOINT = torch.load(r'./checkpoints/temps/model_0')
model = NoteComposeNet()
# model.load_state_dict(CHECKPOINT)

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 0.142MB


In [5]:
CSV_PATH = r'datasets/midi-dataset-mini.csv'
df = pd.read_csv(CSV_PATH)
train_midi = MidiDataset(df, context_len = model._context_len, samples_per_track=TRAIN_SAMPLES_PER_TRACK)
vali_midi = MidiDataset(df, context_len= model._context_len, samples_per_track=VALIDATE_SAMPLES_PER_TRACK)
del df

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.0001,
    weight_decay=0.1, 
    betas=(0.9, 0.95),
    eps=1e-05,
    amsgrad=False
    )

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
scheduler = CosineAnnealingWarmRestarts(
    optimizer=optimizer, 
    T_0=2,
    eta_min=0.0001,
)

In [7]:
pipeline = TrainPipeline(train_midi, vali_midi, model, loss_fn, optimizer, validate=False, batch_size=BATCH_SIZE)
pipeline.train(EPOCHS, start_length=64)

Epoch 1:   0%|          | 0/5 [00:15<?, ?batch/s]


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list